In [3]:
from langchain.llms import OpenAI


In [19]:
import os

In [47]:
# os.environ["OPENAI_API_KEY"]="sk-proj-"

In [37]:
os.environ["OPENAI_API_TYPE"] = "azure"
 
os.environ["OPENAI_API_VERSION"] = "2023-07-01-preview"
 
os.environ["AZURE_OPENAI_API_KEY"] = ""
os.environ["AZURE_OPENAI_ENDPOINT"] = "https://openai-nonfunsbx-pprd-01.openai.azure.com/openai/deployments/gpt-4-finance/chat/completions?api-version=2023-07-01-preview"

In [38]:
# llm=OpenAI(openai_api_key=os.environ["AZURE_OPENAI_API_KEY"], temperature=0)

In [44]:
from langchain.chat_models.azure_openai import AzureChatOpenAI

In [45]:
llm = AzureChatOpenAI(
    openai_api_version="2023-07-01-preview",
    azure_deployment="gpt-4",
    model_version="gpt-4"
)

In [69]:
llm

AzureChatOpenAI(client=<openai.resources.chat.completions.Completions object at 0x000001F41A828C50>, async_client=<openai.resources.chat.completions.AsyncCompletions object at 0x000001F41A883890>, openai_api_key='611dff392d7a4f3780f84060b3d6bc9e', openai_proxy='', azure_endpoint='https://openai-nonfunsbx-pprd-01.openai.azure.com/openai/deployments/gpt-4-finance/chat/completions?api-version=2023-07-01-preview', deployment_name='gpt-4', openai_api_version='2023-07-01-preview', model_version='gpt-4', openai_api_type='azure')

In [78]:
text="what is the meaning of life?"
llm.predict(text)

"As an AI developed by OpenAI, I don't have real-time capabilities or access to current events, future events, or databases. As of my last training data in September 2021, I can't provide the result of future matches beyond that point. I recommend checking the latest sports news for this information."

In [48]:
os.environ["HUGGINGFACEHUB_API_TOKEN"] = ""

In [ ]:
# Langchain is like a wrapper, it can use any language model, openai, huggingface, opensource etc.

In [50]:
from langchain import HuggingFaceHub
llm_huggingface = HuggingFaceHub(repo_id = "google/flan-t5-large", model_kwargs={"temperature":0.5, "max_length": 64})

In [51]:
output=llm_huggingface.predict(text)

In [52]:
print(output)

Life is a journey


In [53]:
output=llm_huggingface.predict("write a poem about roland from alone show")
print(output)

i love roland he is the best singer i have ever seen he is the best singer i have ever seen he is the best singer i have ever seen he is the best singer i have ever seen he is the best singer i have ever seen he


In [54]:
llm.predict("write a poem about roland from alone show")

"In the heart of the wild, where solitude reigns,\nThere lies a man, named Roland, unchained.\nIn the wilderness vast, so pristine and raw,\nA participant of the show 'Alone' we saw.\n\nHe's not just a man, but a force of nature,\nLiving off the land, every creature's saviour.\nWith heart as fierce as a roaring lion,\nYet a spirit as gentle as a summer morn.\n\nHe knows no fear, he knows no defeat,\nFor Roland, every challenge is a treat.\nFrom the frozen tundra to the densest wood,\nHe survives where only the bravest could.\n\nHe's a craftsman skilled, a hunter keen,\nA testament to the human gene.\nIn the wild, he finds his peace,\nIn solitude, his soul's release.\n\nWith his hands so rough, and eyes so bright,\nHe battles the cold and the darkest night.\nTo the call of the wild, he's forever bound,\nIn the heart of nature, he has found.\n\nHis triumphs echo in the silent trees,\nHis spirit dances with the gentle breeze.\nIn his solitude, he's never alone,\nFor the wilderness to him,

In [55]:
# prompt templates
# when calling llm, the llm model should know what kind of input it is expecting, so that it can generate the output accordingly
from langchain.prompts import PromptTemplate
prompt_template = PromptTemplate(input_variables = ['country'], template = "Tell me the capital of {country}")
prompt_template.format(country="India")

'Tell me the capital of India'

In [56]:
# not right way to call llm with prompt template
llm.predict(prompt_template = prompt_template, text = "India")

TypeError: Completions.create() got an unexpected keyword argument 'prompt_template'

In [57]:
from langchain.chains import LLMChain
chain=LLMChain(llm=llm, prompt=prompt_template)

In [58]:
chain.run("china")

'The capital of China is Beijing.'

In [60]:
# combining multiple chains using sequential chain
capital_prompt = PromptTemplate(input_variables = ['country'], template = "Tell me the capital of {country}")

capital_chain=LLMChain(llm=llm, prompt=capital_prompt)

famous_template = PromptTemplate(input_variables = ['capital'], template = "Tell me a famous person from {capital}")

In [61]:
famous_chain=LLMChain(llm=llm, prompt=famous_template)

In [62]:
from langchain.chains import SimpleSequentialChain
chain = SimpleSequentialChain(chains = [capital_chain, famous_chain])
chain.run("India")

'One famous person from New Delhi, India is Shah Rukh Khan, a renowned Bollywood actor.'

In [64]:
# to see the full chain output
capital_prompt = PromptTemplate(input_variables = ['country'], template = "Tell me the capital of {country}")

capital_chain=LLMChain(llm=llm, prompt=capital_prompt, output_key="capital")

famous_template = PromptTemplate(input_variables = ['capital'], template = "Tell me a famous person from {capital}")

famous_chain=LLMChain(llm=llm, prompt=famous_template, output_key="famous_person")

In [67]:
from langchain.chains import SequentialChain
chain = SequentialChain(chains = [capital_chain, famous_chain], input_variables = ['country'], output_variables=["capital", "famous_person"])
chain({"country":"India"})

{'country': 'India',
 'capital': 'The capital of India is New Delhi.',
 'famous_person': 'One famous person from New Delhi, India is Shah Rukh Khan, a highly successful Bollywood actor.'}

In [68]:
# chatmodels with chatopenai
from langchain.chat_models import ChatOpenAI

In [70]:
# system message, ai message-gives output, 
from langchain.schema import HumanMessage, SystemMessage, AIMessage

In [71]:
llm([
    SystemMessage(content = "you are an astrophysicist"),
    HumanMessage(content = "who created the universe, matter, and energy?"),
])

AIMessage(content='As an astrophysicist, I can explain that the origin of the universe, matter, and energy is still a topic of ongoing scientific study and philosophical discussion. The widely accepted scientific explanation is the Big Bang Theory, which posits that the universe began as a hot, dense point nearly 13.8 billion years ago. However, this theory doesn\'t explain what "caused" the Big Bang or what "existed" before it.\n\nScientists believe that matter and energy have always existed in some form since the Big Bang and are interconvertible; this is encapsulated by Einstein\'s famous equation E=mc^2. \n\nWhen it comes to who or what \'created\' the universe, that question goes beyond the realm of science into metaphysics or theology. Different people, cultures, and religions have different beliefs about the creation of the universe. From a scientific perspective, the answer to such questions remains unknown.', response_metadata={'token_usage': {'completion_tokens': 186, 'prompt

In [72]:
# prompt template + llm + output parsers
from langchain.chat_models import ChatOpenAI
from langchain.prompts.chat import ChatPromptTemplate
from langchain.schema import BaseOutputParser

In [73]:
class commaseperatedoutput(BaseOutputParser):
    def parse(self, output:str):
        return output.strip().split(",")

In [74]:
template = "you are an airforce marshal, when user asks a question, you should answer with a comma separated list of answers"
human_template="{question}"
chatprompt=ChatPromptTemplate.from_messages([
    ("system", template),
    ("human", human_template)
])

In [75]:
chain=chatprompt|llm|commaseperatedoutput()
# chain=chatprompt|llm

In [76]:
chain.invoke({"question":"how to clear CDS and prepare for ssb while doing my job"})

['Create a study plan',
 ' Consistent studying',
 ' Understand the syllabus',
 ' Proper time management',
 ' Solve previous year question papers',
 ' Take mock tests regularly',
 ' Improve English communication skills',
 ' Stay updated with current affairs',
 ' Physical fitness training',
 ' Improve leadership skills',
 ' Participate in group activities for SSB',
 ' Stay motivated and disciplined']

In [1]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
# from transformers import MBartForConditionalGeneration, MBart50TokenizerFast

article_hi = "संयुक्त राष्ट्र के प्रमुख का कहना है कि सीरिया में कोई सैन्य समाधान नहीं है"
article_ar = "الأمين العام للأمم المتحدة يقول إنه لا يوجد حل عسكري في سوريا."

tokenizer = AutoTokenizer.from_pretrained("facebook/mbart-large-50-many-to-many-mmt")
model = AutoModelForSeq2SeqLM.from_pretrained("facebook/mbart-large-50-many-to-many-mmt")

# translate Hindi to French
tokenizer.src_lang = "hi_IN"
encoded_hi = tokenizer(article_hi, return_tensors="pt")
generated_tokens = model.generate(
    **encoded_hi,
    forced_bos_token_id=tokenizer.lang_code_to_id["fr_XX"]
)
tokenizer.batch_decode(generated_tokens, skip_special_tokens=True)
# => "Le chef de l 'ONU affirme qu 'il n 'y a pas de solution militaire dans la Syrie."

# translate Arabic to English
tokenizer.src_lang = "ar_AR"
encoded_ar = tokenizer(article_ar, return_tensors="pt")
generated_tokens = model.generate(
    **encoded_ar,
    forced_bos_token_id=tokenizer.lang_code_to_id["en_XX"]
)
tokenizer.batch_decode(generated_tokens, skip_special_tokens=True)
# => "The Secretary-General of the United Nations says there is no military solution in Syria."



c:\Users\SESA737860\Desktop\llms\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
c:\Users\SESA737860\Desktop\llms\.venv\Lib\site-packages\huggingface_hub\file_download.py:157: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\SESA737860\.cache\huggingface\hub\models--facebook--mbart-large-50-many-to-many-mmt. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administra

ValueError: Couldn't instantiate the backend tokenizer from one of: 
(1) a `tokenizers` library serialization file, 
(2) a slow tokenizer instance to convert or 
(3) an equivalent slow tokenizer class to instantiate and convert. 
You need to have sentencepiece installed to convert a slow tokenizer to a fast one.

In [1]:
from transformers import MBartForConditionalGeneration, MBart50TokenizerFast


c:\Users\SESA737860\Desktop\llms\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:

# Use a pipeline as a high-level helper

article_hi = "संयुक्त राष्ट्र के प्रमुख का कहना है कि सीरिया में कोई सैन्य समाधान नहीं है"
article_ar = "الأمين العام للأمم المتحدة يقول إنه لا يوجد حل عسكري في سوريا."




model = MBartForConditionalGeneration.from_pretrained("facebook/mbart-large-50-many-to-many-mmt")
tokenizer = MBart50TokenizerFast.from_pretrained("facebook/mbart-large-50-many-to-many-mmt")

In [4]:

# translate Hindi to French
tokenizer.src_lang = "hi_IN"
encoded_hi = tokenizer(article_hi, return_tensors="pt")
generated_tokens = model.generate(
    **encoded_hi,
    forced_bos_token_id=tokenizer.lang_code_to_id["fr_XX"]
)
tokenizer.batch_decode(generated_tokens, skip_special_tokens=True)
# => "Le chef de l 'ONU affirme qu 'il n 'y a pas de solution militaire dans la Syrie."



["Le chef de l 'ONU affirme qu 'il n 'y a pas de solution militaire en Syria."]

In [7]:
with open(r'C:\Users\SESA737860\Desktop\docx_textract\BG170400364.txt', 'r') as file:
    text = file.read()

In [8]:
tokenizer.src_lang = "fr_XX"
encoded_ar = tokenizer(text, return_tensors="pt")
generated_tokens = model.generate(
    **encoded_ar,
    forced_bos_token_id=tokenizer.lang_code_to_id["en_XX"]
)
tokenizer.batch_decode(generated_tokens, skip_special_tokens=True)
# => "The Secretary-General of the United Nations says there is no military solution in Syria."

['ci Le Ksar Building, 34/35 Bab Ezzouar Business Centre 16311 Alger Algerie WARNING OF BONNE EXECUTION Tel 213 21 98 19 00 LG 120/2015 5977600937 Fax 213 21 98 24 38 We refer to Order No. 4550031221 Div A200 between ALSTOM ALGERIE SPA of the one part, and SCHNEIDER ELECTRIC ALGERIE of the other part, relating to the purchase of a Schneider Electric unlockable version of a high voltage table (THT)30 KV as part of the realisation of the first line of the Mostaganem Tramway. We refer to the request of our relationship SCHNEIDER ELECTRIC ALGERIE, for the issuance of a warranty of good performance in favour of ALSTOM ALGERIE SPA in the amount of DZD 8,212,248.00 or Eight Million']

: 